In [1]:
import numpy as np
from typing import Union

from qiskit import Aer, QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.utils import QuantumInstance
from qiskit.primitives import Sampler, Estimator, BackendSampler, BackendEstimator
from qiskit.opflow import StateFn, CircuitStateFn, PauliSumOp

from squlearn.util.quantum_fisher import get_quantum_fisher

from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.random import random_circuit

from squlearn.util import Executor

from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options

from qiskit_ibm_runtime import Estimator as Estimator_runtime

import logging

logging.basicConfig(level=logging.INFO, filename="qiskit.log", filemode="w")

In [2]:
# Generate executor from different backend / execution models:

# String:
executor = Executor("statevector_simulator")
executor = Executor("qasm_simulator")

# Backend:
executor = Executor(Aer.get_backend("statevector_simulator"))

# Quantum Instance:
QI = QuantumInstance(Aer.get_backend("qasm_simulator"), shots=1000)
executor = Executor(QI)

# Service (for execution on the real backend, uncomment the following lines and replace YOUR_TOKEN_HERE with your token):
# service = QiskitRuntimeService(channel="ibm_quantum",token="YOUR_TOKEN_HERE")
# executor = Executor(service,backend = service.get_backend('ibm_nairobi'))
# # alternatively:
# executor = Executor(service.get_backend('ibm_nairobi'))

# # Session
# session = Session(service, backend=service.get_backend('ibm_nairobi'),max_time=28800)
# executor = Executor(session)

# # Estimator
# estimator = Estimator_runtime(session)
# executor = Executor(estimator)

# Primitive:
executor = Executor(Estimator())
executor = Executor(Sampler())
executor = Executor(BackendEstimator(Aer.get_backend("qasm_simulator")))
executor = Executor(BackendSampler(Aer.get_backend("qasm_simulator"), {"shots": 1000}))

C:\Users\DKR\AppData\Local\Temp\ipykernel_19328\1414278804.py:11: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  QI = QuantumInstance(Aer.get_backend('qasm_simulator'),shots=1000)


In [3]:
# Shots can be set by the executor:
print("Current shots as set before:", executor.get_shots())
# Set shots
executor.set_shots(1234)
print("Adjusted shots :", executor.get_shots())
# Reset shots to initial ones:
executor.reset_shots()
print("Reset shots :", executor.get_shots())

Current shots as set before: 1000
Adjusted shots : 1234
Reset shots : 1000


In [4]:
# The executor generates modified Primitives that can be used as usual:
# The modified primitives allow caching, automatic session handling and logging.

# Generate a random circuit:
circuit = random_circuit(2, 2, seed=0).decompose(reps=1)

# Generate a observable:
observable = SparsePauliOp("ZI")

# Get the Executor Estimator Primitive and call run:
estimator = executor.get_estimator()
print(estimator.run(circuit, observable, shots=4321).result())

# Get the Executor Sampler Primitive and call run:
circuit.measure_all()
sampler = executor.get_sampler()
print(sampler.run(circuit).result())

EstimatorResult(values=array([-0.12149965]), metadata=[{'variance': 0.9852378343553527, 'shots': 4321}])
SamplerResult(quasi_dists=[{3: 0.511, 0: 0.416, 2: 0.038, 1: 0.035}], metadata=[{'shots': 1000}])


In [5]:
# The executor can also be used to execute backend.run() (caching not implemented yet)
job = executor.backend_run(circuit)
job.result()

Result(backend_name='qasm_simulator', backend_version='0.12.0', qobj_id='', job_id='8ed199ec-d01b-422a-8083-e136b64b2d46', success=True, results=[ExperimentResult(shots=1000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x3': 536, '0x0': 403, '0x2': 37, '0x1': 24}), header=QobjExperimentHeader(creg_sizes=[['meas', 2]], global_phase=0.0, memory_slots=2, metadata=None, n_qubits=2, name='circuit-125', qreg_sizes=[['q', 2]]), status=DONE, seed_simulator=480264778, metadata={'batched_shots_optimization': False, 'method': 'statevector', 'active_input_qubits': [0, 1], 'device': 'CPU', 'remapped_qubits': False, 'num_qubits': 2, 'num_clbits': 2, 'sample_measure_time': 0.0003055, 'input_qubit_map': [[0, 0], [1, 1]], 'measure_sampling': True, 'noise': 'ideal', 'parallel_shots': 1, 'parallel_state_update': 12, 'fusion': {'enabled': True, 'threshold': 14, 'max_fused_qubits': 5, 'applied': False}}, time_taken=0.0019038)], date=2023-05-10T11:52:23.584640, status=COMPLETED, header=No

In [6]:
# Opflow structures can be used as well:
circuit = random_circuit(2, 2, seed=0).decompose(reps=1)

opflow = StateFn(PauliSumOp(observable), is_measurement=True) @ CircuitStateFn(circuit)
print(opflow)

result = executor.opflow_exec(opflow)
print(result)

ComposedOp([
  OperatorMeasurement(1.0 * ZI),
  CircuitStateFn(
         ┌──────────┐  ┌────┐┌─────────┐     ┌─────────────┐      ┌──────────┐»
  q_0: ──┤ Rz(-π/2) ├──┤ √X ├┤ Rz(π/2) ├──■──┤ Ry(0.84756) ├───■──┤ Rz(-π/2) ├»
       ┌─┴──────────┴─┐├───┬┘└─────────┘┌─┴─┐├─────────────┴┐┌─┴─┐└─┬─────┬──┘»
  q_1: ┤ Rz(-0.25744) ├┤ S ├────────────┤ X ├┤ Ry(-0.84756) ├┤ X ├──┤ Sdg ├───»
       └──────────────┘└───┘            └───┘└──────────────┘└───┘  └─────┘   »
  «         ┌──────┐   ┌─────────┐┌────────────┐
  «q_0: ────┤ √Xdg ├───┤ Rz(π/2) ├┤ R(5.735,0) ├
  «     ┌───┴──────┴──┐└─────────┘└────────────┘
  «q_1: ┤ Rz(0.25744) ├─────────────────────────
  «     └─────────────┘                         
  )
])
opflow ComposedOp([
  OperatorMeasurement(1.0 * ZI),
  CircuitStateFn(
         ┌──────────┐  ┌────┐┌─────────┐     ┌─────────────┐      ┌──────────┐»
  q_0: ──┤ Rz(-π/2) ├──┤ √X ├┤ Rz(π/2) ├──■──┤ Ry(0.84756) ├───■──┤ Rz(-π/2) ├»
       ┌─┴──────────┴─┐├───┬┘└─────────┘┌─┴─┐├───────

C:\Users\DKR\AppData\Local\Temp\ipykernel_19328\1503346594.py:4: DeprecationWarning: The class ``qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  opflow = StateFn(PauliSumOp(observable),is_measurement=True) @ CircuitStateFn(circuit)
C:\Users\DKR\AppData\Local\Temp\ipykernel_19328\1503346594.py:4: DeprecationWarning: The class ``qiskit.opflow.state_fns.operator_state_fn.OperatorStateFn`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  opflow = StateFn(PauliSumOp(observable),is_measurement=True) @ CircuitStateFn(circuit)
C:\Users\DKR\AppData\Local\Temp\ipykernel_19328\1503346594.py:4: DeprecationWarning: The class ``qiskit.opflow.state_fns.circuit_state_fn.CircuitStateFn``

In [7]:
# The executor features a log of what is done in the background (useful for real backends):

executor = Executor(BackendSampler(Aer.get_backend("qasm_simulator")), log_file="example_log.log")
executor.set_shots(1234)
estimator = executor.get_estimator()
print(estimator.run(circuit, observable, shots=4321).result())

EstimatorResult(values=array([-0.12288822]), metadata=[{'variance': 0.9848984853067378, 'shots': 4321}])


In [8]:
# The executor features a cache where the result of jobs are stored and can be reused:

executor = Executor(
    BackendSampler(Aer.get_backend("qasm_simulator")),
    log_file="example_log_cache.log",
    caching=True,
    cache_dir="_cache",
)
executor.set_shots(1234)
estimator = executor.get_estimator()
print(estimator.run(circuit, observable, shots=4321).result())
print(estimator.run(circuit, observable, shots=1234).result())
print(estimator.run(circuit, observable, shots=4321).result())

EstimatorResult(values=array([-0.14001389]), metadata=[{'variance': 0.9803961118182966, 'shots': 4321}])
EstimatorResult(values=array([-0.13938412]), metadata=[{'variance': 0.9805720680135228, 'shots': 1234}])
EstimatorResult(values=array([-0.14001389]), metadata=[{'variance': 0.9803961118182966, 'shots': 4321}])
